In [1]:
# Importing the libraries
from pathlib import Path

import pandas as pd
import numpy as np
#from tensorflow import keras
#from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [ ]:
base_csv = Path('data/clean_frame.csv')

In [423]:
base_df = pd.read_csv(base_csv, index_col='ind')
base_df.columns

Index(['Код_группы', 'Год_Поступления', 'Пособие', 'Общежитие',
       'Наличие_Матери', 'Наличие_Отца', 'Опекунство', 'Село', 'Иностранец',
       'КодФакультета', 'СрБаллАттестата', 'Статус', 'male', 'female',
       'birth_year_int', 'basis', 'language', 'country', 'region', 'city',
       'parents_country'],
      dtype='object')

In [424]:
base_df = base_df.fillna(0)

In [425]:
columns = ['Наличие_Матери', 'Село', 'Иностранец', 'СрБаллАттестата', 'Статус', 'male', 'basis','language','country', 'КодФакультета', 'Общежитие', 'parents_country']
#columns = base_df.columns.to_list()
priznaki = len(columns)
priznaki

12

In [426]:
frame_to_train = base_df.loc[:, columns]

In [427]:
result_key = {
    -1: 0,
    3: 1,
    4: 2,
}

In [428]:
for key, value in result_key.items():
    frame_to_train.loc[frame_to_train['Статус'] == key, 'Статус'] = value

frame_to_train.groupby('Статус').count()

,Наличие_Матери,Село,Иностранец,СрБаллАттестата,male,basis,language,country,КодФакультета,Общежитие,parents_country
Статус,,,,,,,,,,,
0,603,603,603,603,603,603,603,603,603,603,603
1,4660,4660,4660,4660,4660,4660,4660,4660,4660,4660,4660
2,8181,8181,8181,8181,8181,8181,8181,8181,8181,8181,8181


In [429]:
train, test = train_test_split(frame_to_train, test_size=0.20)

In [430]:
train_input = train.drop('Статус', axis=1)
train_output = train['Статус']

In [431]:
test_input = test.drop('Статус', axis=1)
test_output = test['Статус']

In [432]:
train_output.info

<bound method Series.info of ind
5330    2
1409    1
7708    1
6959    2
2935    2
       ..
9618    2
5634    2
1506    2
2662    2
6530    2
Name: Статус, Length: 10755, dtype: int64>

In [433]:
input_df_size = train_input.shape[0]

In [434]:
# Constructing the input
x = train_input.to_numpy()
y = train_output.values
arr = np.transpose(x)

In [435]:
output = np.ones((1,input_df_size))
output[0] = y
arr_y = output.reshape(input_df_size,1)
arr_y

array([[2.],
       [1.],
       [1.],
       ...,
       [2.],
       [2.],
       [2.]])

In [436]:
# Definig the model
model_better = keras.Sequential([
    keras.layers.Dense(150, input_shape=(priznaki-1,), activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(250, activation='relu'),    
    keras.layers.Dense(3, activation='softmax'),
])

In [437]:
# Compiling the model
model_better.compile(optimizer='Nadam',
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])

In [438]:
# fitting the model
model_better.fit(x, y, epochs=100, batch_size=70)

Epoch 1/100
154/154 [==============================] - 1s 839us/step - loss: 0.8037 - accuracy: 0.6077
Epoch 2/100
154/154 [==============================] - 0s 848us/step - loss: 0.7824 - accuracy: 0.6206
Epoch 3/100
154/154 [==============================] - 0s 840us/step - loss: 0.7683 - accuracy: 0.6320
Epoch 4/100
154/154 [==============================] - 0s 830us/step - loss: 0.7597 - accuracy: 0.6421
Epoch 5/100
154/154 [==============================] - 0s 849us/step - loss: 0.7532 - accuracy: 0.6461
Epoch 6/100
154/154 [==============================] - 0s 921us/step - loss: 0.7427 - accuracy: 0.6536
Epoch 7/100
154/154 [==============================] - 0s 824us/step - loss: 0.7377 - accuracy: 0.6576
Epoch 8/100
154/154 [==============================] - 0s 891us/step - loss: 0.7308 - accuracy: 0.6623
Epoch 9/100
154/154 [==============================] - 0s 1ms/step - loss: 0.7258 - accuracy: 0.6663
Epoch 10/100
154/154 [==============================] - 0s 1ms/step - loss:

In [439]:
# Evaluating the model
x_t = test_input.to_numpy()
y_t = test_output.values

model_better.evaluate(x_t, y_t, batch_size=8)

337/337 [==============================] - 0s 526us/step - loss: 0.6803 - accuracy: 0.7047


[0.6803385019302368, 0.7047229409217834]

In [440]:
tusa = model_better.predict(x_t)

85/85 [==============================] - 0s 479us/step


In [441]:
def get_result(answer_arr: np.array) -> int:
    answer_code = np.argmax(answer_arr)

    # list out keys and values separately
    key_list = list(result_key.keys())
    val_list = list(result_key.values())
    
    # print key with val 100
    position = val_list.index(answer_code)
    return  key_list[position], answer_code

In [442]:
answers = []
for answer_arr in tusa:
    answers.append(get_result(answer_arr)[1])

In [443]:
test_output

ind
6125     2
5803     2
976      2
5452     2
4474     2
        ..
13042    2
8030     2
10807    1
11190    2
6085     1
Name: Статус, Length: 2689, dtype: int64

In [444]:
compare_df = pd.DataFrame(columns=['real', 'calculation'])
compare_df['real'] = y_t
compare_df['calculation'] = answers
compare_df

,real,calculation
0,2,2
1,2,2
2,2,2
3,2,1
4,2,2
...,...,...
2684,2,2
2685,2,2
2686,1,2
2687,2,2


In [445]:
compare_df['new'] = np.where((compare_df['real'] == compare_df['calculation']), compare_df['real'], 100)
compare_df.groupby('new').count()

,real,calculation
new,,
0,2,2
1,493,493
2,1400,1400
100,794,794


In [446]:
r = 1 + 503 + 1418 + 767
767 / r

0.28523614726664187